Visualize and Export simulation data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append("../include/")
import hhtools
import hhsignal

import importlib
from tqdm.notebook import tqdm
importlib.reload(hhtools)

In [ ]:
# Load data
def load(fname):
    import pickle as pkl
    with open(fname, "rb") as fp:
        return pkl.load(fp)


obj = load("./simulation_data/orders.pkl")["obj"]
order_dyna = load("./simulation_data/orders_dyna.pkl")["order_dyna"]

print(obj.summary["chi"].shape, "\n", order_dyna.keys())

## function define

In [ ]:
import itertools
titles = itertools.cycle(iter(["T", "F", "S"]))


def draw_orders(im_set, x=None, y=None, xt=None, yt=None, axs=None, clim=None, alpha_map=None, ftitle=None, title_fontsize=None):
    if alpha_map is None:
        alpha_map = [None, None, None]
    
    pallete = []
    for n in range(3):
        if axs is None:
            plt.subplot(1,3,n+1)
        else:
            plt.axes(axs[n])
        p = hhtools.imshow_xy(im_set[n], x=x, y=y, interpolation="none", clim=clim, cmap="jet", alpha=alpha_map[n])
        pallete.append(p)
        plt.xticks(xt)
        plt.yticks(yt)
        plt.title(ftitle(n), fontsize=title_fontsize)
        plt.colorbar()
    return pallete

def concat_image(order_dyna, key, idr, idp):
    im_avg_set = np.array([order_dyna[key]["mean"][:,:,idr,idp,n] for n in range(3)])
    im_std_set = np.array([order_dyna[key]["std"][:,:,idr,idp,n] for n in range(3)])
    return im_avg_set, im_std_set

def gen_full_figure(obj, order_dyna, id_r, id_p, w=15, dpi=120, vmin_p=0.1, vmax_p=0.8, show_diff=False, show_only_std=False, apply_alpha=False, pts=None, save_dir=None):
    from collections import OrderedDict
    
    # need to check option
    fs = 2000
    
    x = obj.controls["beta_set"]
    y = obj.controls["alpha_set"]
    xt = [0, 0.5, 1]
    yt = [0.5*i for i in range(5)]
    title_fontsize = 15
    
    if show_only_std:
        prefix = "\sigma"
        inv_prefix = "\sigma"
    elif show_diff:
        prefix = "\Delta E"
        inv_prefix = "\Delta (1/E)"
    else:
        prefix = "E"
        inv_prefix = "1/E"
    
    def draw_sub(im_set, im_std, **opt):
        if np.shape(im_set)[2] == 3:
            im_set = [im_set[:,:,n] for n in range(3)]
            im_std = [im_std[:,:,n] for n in range(3)]
            
        alpha_maps = None
        if im_std is not None:
            if apply_alpha:
                alpha_maps = [get_alpha_map(im_std[n], vmin_p=vmin_p, vmax_p=vmax_p) for n in range(3)]
            elif show_only_std:
                im_set = im_std
                opt["clim"] = [None, None]
        
        if show_diff:
            im_set = [im - im[0, 0] for im in im_set]
            
        return draw_orders(im_set, alpha_map=alpha_maps, x=x, y=y, xt=xt, yt=yt, title_fontsize=title_fontsize, **opt), im_set
        
    def get_orders(key):
        data = obj.summary[key][:,:,id_r,id_p,:,:]
        data_avg = np.nanmean(data, axis=2)
        data_std = np.nanstd(data, axis=2)
        return data_avg, data_std
    
    def draw_single(im, ax0, im_std=None, clim=None, title=None, cmap="jet", show_diff=False):
        if clim is None:
            clim = [None, None]
        
        alpha_map = None
        if im_std is not None:
            if apply_alpha:
                alpha_map = get_alpha_map(im_std, vmin_p=vmin_p, vmax_p=vmax_p)
            elif show_only_std:
                im = im_std
                clim = [None, None]
        
        if show_diff:
            im = im - im[0, 0]
        
        plt.axes(ax0)
        im_ax = hhtools.imshow_xy(im, x=x, y=y, vmax=clim[1], vmin=clim[0], alpha=alpha_map, cmap=cmap)
        plt.title(title, fontsize=title_fontsize)
        plt.xticks(xt)
        plt.yticks(yt)
        cbar = plt.colorbar()
        
        return im_ax, cbar, im
    
    def get_cl(clim):
        clim = np.array(clim)
        if show_diff:
            clim = clim - np.average(clim)
        return clim
        
    import itertools
    titles = itertools.cycle(iter(["T", "F", "S"]))
    
    summary_data = OrderedDict()
    fig, axs = plt.subplots(7, 4, dpi=dpi, figsize=(w, w*np.sqrt(2)))
    
    # ---------------------  Firing rate ---------------------
    fr_avg, fr_std = get_orders("frs_m")
    cl = get_cl([0, 10])
    _, im = draw_sub(fr_avg, fr_std, axs=axs[0][:3], clim=cl, ftitle=lambda x: r"$%s[z(%s)]$"%(prefix, next(titles)))
    summary_data["frs_m"] = im
    
    # ---------------------  Chi ---------------------
    chi_avg, chi_std = get_orders("chi")
    cl = get_cl([0, 0.2+0.6*id_r]) 
    _, im = draw_sub(chi_avg, chi_std, axs=axs[1][:3], clim=cl, ftitle=lambda x: r"$%s[\chi(%s)]$"%(prefix, next(titles)))
    summary_data["chi"] = im
    
    # --------------------- CV -----------------------
    cv_avg, _ = get_orders("cv")
    _, _, cv1 = draw_single(cv_avg[:,:,1], axs[0][3], clim=(0, 1), title=r"$E[CV(F)]$", show_diff=False)
    _, _, cv2 = draw_single(cv_avg[:,:,2], axs[1][3], clim=(0, 1), title=r"$E[CV(S)]$", show_diff=False)
    summary_data["cv"] = [cv1, cv2]
    
    # ---------------------  AC2_D ---------------------
    ac2d_avg, ac2d_std = concat_image(order_dyna, "ac2p_1st", id_r, id_p)
    clim = get_cl([0, 0.5+0.3*id_r])
    _, im = draw_sub(ac2d_avg, ac2d_std, axs=axs[2][:3], clim=clim, ftitle=lambda x: r"$%s[\bar{A}_1(%s)]$"%(prefix, next(titles)))
    summary_data["ac2p_1st"] = im
    
    # ---------------------  AC2_P ---------------------
    ac2d_avg, ac2d_std = concat_image(order_dyna, "ac2p_large", id_r, id_p)
    _, im = draw_sub(ac2d_avg, ac2d_std, axs=axs[3][:3], clim=clim, ftitle=lambda x: r"$%s[\bar{A}_{large}(%s)]$"%(prefix, next(titles)))
    summary_data["ac2p_large"] = im
    
    # ---------------------  tau_D ---------------------    
    tau_avg_1st, tau_std =concat_image(order_dyna, "ac2t_1st", id_r, id_p)
    clim = get_cl([20, 50+30*id_r])
    _, im = draw_sub(1/tau_avg_1st, tau_std, axs=axs[4][:3], clim=clim, ftitle=lambda x: r"$%s[\bar{\tau}_1(%s)]$"%(inv_prefix, next(titles)))
    summary_data["1/tau_1st"] = im
    
    # ---------------------  tau_P ---------------------
    tau_avg_large, tau_std = concat_image(order_dyna, "ac2t_large", id_r, id_p)
    _, im = draw_sub(1/tau_avg_large, tau_std, axs=axs[5][:3], clim=clim, ftitle=lambda x: r"$%s[\bar{\tau}_{large}(%s)]$"%(inv_prefix, next(titles)))
    summary_data["1/tau_large"] = im
    
    # ---------------------  delta tau ---------------------
    del_t = 1/tau_avg_1st - 1/tau_avg_large
    clim = get_cl([0, 20*(id_r+1)])
    draw_sub(del_t, None, axs=axs[6][:3], clim=clim, ftitle=lambda x: r"$1/E[{\tau}_1]-1/E[{\tau}_{large}](%s)$"%(next(titles)))
    
    
    # ---------------------  CC ---------------------
    cc_avg = order_dyna["cc1p_large"]["mean"][:,:,id_r,id_p]
    cc_std = order_dyna["cc1p_large"]["std"][:,:,id_r,id_p]
    _, _, im = draw_single(cc_avg, axs[3][3], im_std=cc_std, clim=(0, 0.8+0.2*id_r), title=r"$%s[\bar{C}]$"%(prefix))
    summary_data["cc1p"] = im
    
    # ---------------------  Ratio of Leading ---------------------
    tau_cc = order_dyna["cc1t_large"][:,:,id_r,id_p,:]
    # remove which have only 1 point differnce
    ind = np.abs(tau_cc * fs) < 2
    tau_cc[ind] = 0
    
    pos_ratio = np.sum(tau_cc > 0, axis=2) / tau_cc.shape[2]
    pos_ratio -= np.sum(tau_cc < 0, axis=2) / tau_cc.shape[2]
    # pos_ratio[pos_ratio == 0] = np.nan
    _, cbar, im = draw_single(pos_ratio, axs[4][3], clim=(-1, 1), title=r"$\eta$", show_diff=False)
    cbar.ax.set_yticks([-1, 0, 1])
    cbar.ax.set_yticklabels(['1 (F>)', '0', '1 (S>)'])
    summary_data["leading_ratio"] = pos_ratio
    
    # ---------------------  Sum of leading case ---------------------
    sum_ratio = np.sum(tau_cc > 0, axis=2) / tau_cc.shape[2]
    sum_ratio += np.sum(tau_cc < 0, axis=2) / tau_cc.shape[2]
    # sum_ratio[sum_ratio == 0] = np.nan
    _, cbar, im = draw_single(sum_ratio, axs[5][3], clim=(0, 1), title=r"$|\eta|$", show_diff=False)
    cbar.ax.set_yticks([0, 0.5, 1])
    summary_data["leading_ratio(abs)"] = sum_ratio
    
    # ---------------------  Delta Phi ---------------------
    tau_cc_avg = np.average(tau_cc, axis=2)
    id_u = pos_ratio > 0.05
    id_d = pos_ratio < -0.05
    
    dphi = np.zeros(tau_cc.shape[:2])
    dphi[id_u] = tau_cc_avg[id_u] / tau_avg_large[2][id_u]
    dphi[id_d] = -tau_cc_avg[id_d] / tau_avg_large[1][id_d]
    # dphi[np.isnan(pos_ratio)] = np.nan
    
    _, cbar, im = draw_single(dphi, axs[6][3], clim=(0, 0.5), title=r"$E[t_{lag,cc}]/E[T_{leader}]$", show_diff=False)
    cbar.ax.set_yticks([0, 0.2, 0.4])
    summary_data["dphi"] = dphi
    
    # ---------------------  White board ---------------------
    im = np.zeros([len(y), len(x), 3])
    im[:,:,1:] = 0.5
    _, cbar, im = draw_single(im, axs[2][3], show_diff=False)
    cbar.ax.set_yticks([])
    plt.xlabel(r"$\beta$", fontsize=15)
    plt.ylabel(r"$\alpha$", fontsize=15)
    axs[2][3].xaxis.set_label_coords(.5, .15)
    axs[2][3].yaxis.set_label_coords(.15, .5)
    
    # ---------------------  pts ---------------------
    cmap = hhtools.get_palette("gray")
    if pts is not None:
        for i in range(len(axs)):
            for j in range(len(axs[0])):
                for n, r in enumerate(pts):
                    nr, nc = r[0], r[1]
                    c = cmap((n+0.5)/len(pts))
                    axs[i][j].plot(x[nc], y[nr], "p", c=c, markersize=5)
    
        for n, r in enumerate(pts):
            nr, nc = r[0], r[1]
            c = cmap((n+0.5)/len(pts))
            axs[2][3].text(x[nc], y[nr]+0.05, "%d"%(n), c=c, fontsize=14, horizontalalignment="center")
    
    # ----------- super title and save
    rr = obj.controls["rank_set"][id_r]
    pr = obj.controls["p_ratio_set"][id_p]
    plt.suptitle("rank=%d, pr=%.2f"%(rr, pr), fontsize=title_fontsize+5)
    
    plt.tight_layout()
    
    if save_dir is not None:
        if show_only_std:
            tag = "std"
        elif apply_alpha:
            tag = "alpha"
        elif show_diff:
            tag = "davg"
        else:
            tag = "avg"
        
        
        fig_name = os.path.join(save_dir, "summary_rank%.1f(%d)_pr%.2f(%d)_%s"%(rr, id_r, pr, id_p, tag))
        
        if pts is not None:
            fig_name += "_pts"
        
        plt.savefig(fig_name+".pdf", format="pdf", dpi=150, backend="Cairo")
    
    # arange axis
    for key in summary_data.keys():
        im = summary_data[key]
        if len(im) <= 3:
            im = np.swapaxes(im, 0, 1)
            im = np.swapaxes(im, 1, 2)
        summary_data[key] = im
    
    
    return summary_data

# Get figures

In [ ]:
from collections import OrderedDict
import pickle as pkl
import datetime

fs = 2000
save_fig_dir = "./simulation_data/figs"

summary_data = OrderedDict()
summary_data_var = OrderedDict()
for idr in range(3): # rank 0, 0.5, 1
    for idp in range(7):
        summary_data["nr%dnp%d"%(idr, idp)] = gen_full_figure(obj, order_dyna, idr, idp, dpi=80,
                                                              vmin_p=0.2, vmax_p=0.8, apply_alpha=False,
                                                              show_only_std=False, save_dir=save_fig_dir, show_diff=False)
        summary_data_var["nr%dnp%d"%(idr, idp)] = gen_full_figure(obj, order_dyna, idr, idp, dpi=80,
                                                                  vmin_p=0.2, vmax_p=0.8, apply_alpha=False,
                                                                  show_only_std=True, save_dir=save_fig_dir, show_diff=False)

with open("./simulation_data/collected_data.pkl", "wb") as fid:
    today = datetime.date.today()
    date = "%d-%d-%d"%(today.year, today.month, today.day)
    
    orders = {"controls": obj.controls,
              "control_names": obj.control_names,
              "data": obj.summary}
    
    pkl.dump({"order": orders,
              "order_dyna": order_dyna,
              "summary_data": summary_data,
              "summary_data_var": summary_data_var,
              "date": date}, fid)